# 🚀 PromptOps with LangSmith and Azure OpenAI

This Jupyter Notebook provides a step-by-step guide on implementing **Prompt Operations (PromptOps)** using **LangSmith** for prompt management, **LangChain** for orchestration, and **Azure OpenAI** for content generation. We will cover the entire lifecycle: prompt registration, modification, loading, generation and deletion.

## Prerequisites

Before running the code, ensure you have the following environment variables set up. These are crucial for connecting to LangSmith and Azure OpenAI.

| Variable | Purpose | Example Value |
| :--- | :--- | :--- |
| `LANGSMITH_API_KEY` | Authentication for LangSmith | `lsv2_sk_...` |
| `LANGSMITH_TRACING` | Name of the LangSmith project for tracing | `true` |
| `AZURE_OPENAI_ENDPOINT` | Your Azure OpenAI Service endpoint | `https://your-resource.openai.azure.com/` |
| `AZURE_OPENAI_API_KEY` | Your Azure OpenAI API key | `abcdef123456...` |
| `OPENAI_API_VERSION` | Azure OpenAI API version | `2024-02-01` |

The Azure OpenAI deployment name used in this tutorial is **`gpt-4.1-mini`**, as specified in the requirements.

TO get LangSmith API key, sign up at [LangSmith](https://langsmith.langchain.com/signup).
## Setup
1. Navigate to settings from bottom left corner.
2. Click on "API Keys" from the left menu.
3. Click on "Create New Key" button to generate a new API key.

In [1]:
!pip install langchain langchain-openai langchain-core langsmith evaluate openai python-dotenv rouge_score --quiet

In [4]:
# 1. Setup and Imports
import os
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
#from evaluate import load
import json

In [ ]:
# --- Environment Variable Setup (Replace with your actual values or set them in your environment) ---
os.environ["LANGSMITH_API_KEY"] = "xxxxxxxxxxxxxxx"
os.environ["LANGSMITH_TRACING"] = "true"


In [6]:
# Initialize LangSmith Client
client = Client()
print("LangSmith Client Initialized.")

LangSmith Client Initialized.


In [7]:
# Initialize Azure OpenAI Model
try:
    llm = AzureChatOpenAI(
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        azure_deployment="gpt-4o-mini",
        temperature=0.7
    )
    print("AzureChatOpenAI Model Initialized with deployment: gpt-4.1-mini")
except Exception as e:
    print(f"Warning: Could not initialize AzureChatOpenAI. Please ensure all environment variables are set correctly. Error: {e}")
    llm = None # Set to None if initialization fails


AzureChatOpenAI Model Initialized with deployment: gpt-4.1-mini


## 1. Prompt Registration (Pushing a Prompt)

LangSmith allows you to register and version your prompts programmatically using the `client.push_prompt()` method. We will create a simple prompt for generating short, professional summaries of technical concepts.

The prompt name we will use is **`technical-summary-generator`**.

In [8]:
# Define the initial prompt (Version 1)
prompt_name = "technical-summary-generator"
initial_template = (
    "You are a professional technical writer. "
    "Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. "
    "The summary must be easy to understand for a non-technical audience."
)

initial_prompt = ChatPromptTemplate.from_template(initial_template)

# Push the initial prompt to LangSmith
try:
    url = client.push_prompt(prompt_name, object=initial_prompt)
    print(f"Successfully registered prompt '{prompt_name}' (Version 1).")
    print(f"View in LangSmith: {url}")
except Exception as e:
    print(f"Error pushing prompt: {e}")


Successfully registered prompt 'technical-summary-generator' (Version 1).
View in LangSmith: https://smith.langchain.com/prompts/technical-summary-generator/4fe9f938?organizationId=e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073


## 2. Prompt Modification (Pushing a New Version)

To modify the prompt, we simply define a new version and use `client.push_prompt()` again with the same name. LangSmith automatically handles the versioning, creating a new commit for the change.

**Modification**: We will add a constraint to include a real-world example in the summary.

In [9]:
# Define the modified prompt (Version 2)
modified_template = (
    "You are a professional technical writer. Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. "
    "The summary must be easy to understand for a non-technical audience and **must include a simple, real-world example**."
)

modified_prompt = ChatPromptTemplate.from_template(modified_template)

# Push the modified prompt to LangSmith
try:
    url = client.push_prompt(prompt_name, object=modified_prompt)
    print(f"Successfully updated prompt '{prompt_name}' (Version 2).")
    print(f"View in LangSmith: {url}")
except Exception as e:
    print(f"Error pushing modified prompt: {e}")


Successfully updated prompt 'technical-summary-generator' (Version 2).
View in LangSmith: https://smith.langchain.com/prompts/technical-summary-generator/0b2d114c?organizationId=e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073


## 3. Loading the Prompt (Pulling a Prompt)

We can load the latest version of the prompt using `client.pull_prompt()`. This ensures our application always uses the most up-to-date, tested prompt from the hub.

In [10]:
# Pull the latest prompt from LangSmith
try:
    latest_prompt = client.pull_prompt(prompt_name)
    print(f"Successfully pulled latest prompt: '{prompt_name}'")
    print("\n--- Pulled Prompt Template ---")
    print(latest_prompt.messages[0].prompt.template)
except Exception as e:
    print(f"Error pulling prompt: {e}")
    latest_prompt = None


Successfully pulled latest prompt: 'technical-summary-generator'

--- Pulled Prompt Template ---
You are a professional technical writer. Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. The summary must be easy to understand for a non-technical audience and **must include a simple, real-world example**.


## 4. Content Generation with Azure OpenAI

Now we combine the loaded prompt with our `AzureChatOpenAI` model to create a simple LangChain Expression Language (LCEL) chain for content generation. The execution of this chain will automatically be traced and logged in LangSmith, linking the generated output back to the specific prompt version used.

In [11]:
if llm and latest_prompt:
    # Create the LCEL chain: Prompt -> LLM -> Output Parser
    chain = latest_prompt | llm | StrOutputParser()

    # Define the input concept
    input_concept = "Quantum Computing"

    print(f"Generating summary for: {input_concept}...")

    # Invoke the chain
    try:
        response = chain.invoke({"concept": input_concept})
        print("\n--- Generated Summary ---")
        print(response)
        print("\nCheck LangSmith for the trace of this run!")
    except Exception as e:
        print(f"Error during content generation: {e}")
        response = "Generation Failed"
else:
    response = "Generation Skipped due to setup errors."
    print(response)


Generating summary for: Quantum Computing...

--- Generated Summary ---
Quantum computing is an advanced type of computing that uses the principles of quantum mechanics to process information in ways that traditional computers cannot. Unlike classical computers, which use bits (0s and 1s) to store and manipulate data, quantum computers use qubits, which can represent both 0 and 1 simultaneously due to a property called superposition. This ability allows quantum computers to solve complex problems much faster than classical computers. For example, imagine trying to find the fastest route through a city with many roads; a classical computer would check each route one by one, while a quantum computer could evaluate multiple routes at the same time, significantly speeding up the process.

Check LangSmith for the trace of this run!


## 5. List, delete, and like prompts

You can also list, delete, and like/unlike prompts using the list prompts, delete prompt, like prompt and unlike prompt methods.

In [12]:
# Define the initial prompt (Version 1)
prompt_name = "summary-generator"
initial_template = (
    "You are a professional Summary writer. "
    "Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. "
)

initial_prompt = ChatPromptTemplate.from_template(initial_template)

# Push the initial prompt to LangSmith
try:
    url = client.push_prompt(prompt_name, object=initial_prompt)
    print(f"Successfully registered prompt '{prompt_name}' (Version 1).")
    print(f"View in LangSmith: {url}")
except Exception as e:
    print(f"Error pushing prompt: {e}")


Successfully registered prompt 'summary-generator' (Version 1).
View in LangSmith: https://smith.langchain.com/prompts/summary-generator/ffb726a6?organizationId=e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073


In [13]:
# Define the initial prompt (Version 1)
prompt_name = "anshu-personal-prompt"
initial_template = (
    "You are a professional technical writer. "
    "Your task is to write a concise, one-paragraph summary of the following technical concept: {concept}. "
    "The summary must be easy to understand for a non-technical audience."
)

initial_prompt = ChatPromptTemplate.from_template(initial_template)

# Push the initial prompt to LangSmith
try:
    url = client.push_prompt(prompt_name, object=initial_prompt)
    print(f"Successfully registered prompt '{prompt_name}' (Version 1).")
    print(f"View in LangSmith: {url}")
except Exception as e:
    print(f"Error pushing prompt: {e}")


Successfully registered prompt 'anshu-personal-prompt' (Version 1).
View in LangSmith: https://smith.langchain.com/prompts/anshu-personal-prompt/4fe9f938?organizationId=e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073


In [21]:
# List all prompts in my workspace
prompts = client.list_prompts()
prompts.repos[:5]

[Prompt(repo_handle='anshu-personal-prompt', description='', readme='', id='355d1aff-b5f4-4ae1-b072-a4279066a819', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 32, 32, 470632), updated_at=datetime.datetime(2025, 11, 19, 10, 32, 34, 252930), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner='anshupandey', full_name='anshupandey/anshu-personal-prompt', num_likes=0, num_downloads=0, num_views=0, liked_by_auth_user=False, last_commit_hash='4fe9f938e7e4a8e6e3a89e4d9f4fe00e0b3c917aeae2b6c401f181143201e4e3', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None),
 Prompt(repo_handle='summary-generator', description='', readme='', id='70882c3c-579e-4ff9-88b0-e548380ffccb', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 32, 18, 870314), updated_at=datetime.datetime(2025, 11, 19, 10, 32, 20, 148229), is_public

In [24]:
# List my private prompts that include "joke"
prompts = client.list_prompts(query="summarize", is_public=False)
prompts.repos

[Prompt(repo_handle='summary-generator', description='', readme='', id='70882c3c-579e-4ff9-88b0-e548380ffccb', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 32, 18, 870314), updated_at=datetime.datetime(2025, 11, 19, 10, 32, 20, 148229), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner='anshupandey', full_name='anshupandey/summary-generator', num_likes=0, num_downloads=0, num_views=0, liked_by_auth_user=False, last_commit_hash='ffb726a621c3fb02f28b41b0ff6e2cb02f5b4161ec594c7d92c2520781e98848', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None),
 Prompt(repo_handle='technical-summary-generator', description='', readme='', id='8adedbf4-97d7-4e8a-b28e-5c8c27e57305', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 24, 52, 909509), updated_at=datetime.datetime(2025, 11, 19, 10, 26, 28, 999727), is_publ

In [25]:
prompts = client.list_prompts(is_public=False)
prompts.repos[:5]

[Prompt(repo_handle='anshu-personal-prompt', description='', readme='', id='355d1aff-b5f4-4ae1-b072-a4279066a819', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 32, 32, 470632), updated_at=datetime.datetime(2025, 11, 19, 10, 32, 34, 252930), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner='anshupandey', full_name='anshupandey/anshu-personal-prompt', num_likes=0, num_downloads=0, num_views=0, liked_by_auth_user=False, last_commit_hash='4fe9f938e7e4a8e6e3a89e4d9f4fe00e0b3c917aeae2b6c401f181143201e4e3', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None),
 Prompt(repo_handle='summary-generator', description='', readme='', id='70882c3c-579e-4ff9-88b0-e548380ffccb', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 32, 18, 870314), updated_at=datetime.datetime(2025, 11, 19, 10, 32, 20, 148229), is_public

In [26]:
# Delete a prompt
prompt_name = "anshu-personal-prompt"
client.delete_prompt(prompt_name)

In [27]:
prompts = client.list_prompts(is_public=False)
prompts.repos[:5]

[Prompt(repo_handle='summary-generator', description='', readme='', id='70882c3c-579e-4ff9-88b0-e548380ffccb', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 32, 18, 870314), updated_at=datetime.datetime(2025, 11, 19, 10, 32, 20, 148229), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner='anshupandey', full_name='anshupandey/summary-generator', num_likes=0, num_downloads=0, num_views=0, liked_by_auth_user=False, last_commit_hash='ffb726a621c3fb02f28b41b0ff6e2cb02f5b4161ec594c7d92c2520781e98848', num_commits=1, original_repo_full_name=None, upstream_repo_full_name=None),
 Prompt(repo_handle='technical-summary-generator', description='', readme='', id='8adedbf4-97d7-4e8a-b28e-5c8c27e57305', tenant_id='e5ffb10d-d7d5-5a52-9eb0-1e8ca168f073', created_at=datetime.datetime(2025, 11, 19, 10, 24, 52, 909509), updated_at=datetime.datetime(2025, 11, 19, 10, 26, 28, 999727), is_publ

In [ ]:
p1 = prompts.repos[0].repo_handle
p2 = prompts.repos[1].repo_handle

In [28]:
# Like a prompt
client.like_prompt("efriis/my-first-prompt")

{'likes': 2}

In [29]:
# Unlike a prompt
client.unlike_prompt("efriis/my-first-prompt")

{'likes': 1}

## Conclusion

This tutorial demonstrated a complete PromptOps workflow:

1.  **Registration & Modification**: Using `client.push_prompt()` to version control prompts in LangSmith.
2.  **Loading**: Using `client.pull_prompt()` to ensure the application uses the latest, approved version.
3.  **Generation**: Integrating the prompt with `AzureChatOpenAI` via a LangChain LCEL chain, with automatic tracing to LangSmith.

This workflow is the foundation for robust, and iterative development of LLM applications.